In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from functions import VM
import os
import numpy as np

# Take the sign names in our collections data

In [2]:
signs = sorted(os.listdir("./Keypoint_Data/")) # take all the signs used in data collection
sign_id = {label: num for num, label in enumerate(signs)} # assign a value for each sign (each sign is assign to its index)

In [3]:
signs[0]

'hello'

In [4]:
sign_id['hello']

0

# Create the x and y data


In [5]:
sequences, labels = [], [] # sequences represents our x data while labels represents our y data
# create a loop that creates our x and y data
for sign in signs:
    for sequence in range(VM.seq):
        window = []
        for frames in range(VM.seq_lenght):
            res = np.load(os.path.join(VM.DATA_PATH, sign, str(sequence), f"{frames}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(sign_id[sign])
        
# store our x and y data to a numpy array
x_data = np.array(sequences) 
y_data = to_categorical(labels).astype(int) 

# setup our training and testing partition
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05)

# Build and train the model


In [24]:
x_train.shape

(950, 30, 1662)

In [25]:
# build the model
model = Sequential()
model.add(LSTM(64, return_sequences="True", activation="relu", input_shape=(30,1662)))
model.add(LSTM(128, return_sequences="True", activation="relu"))
model.add(LSTM(64, return_sequences="False", activation="relu"))
model.add(Flatten(input_shape=(x_train.shape[1:])))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(np.array(signs).shape[0], activation="softmax"))

In [26]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"]) # compiler 

In [ ]:
model.fit(x_train, y_train, epochs=1000) # run this line when data collection is done (this will train the model)

Epoch 1/1000
30/30 [==============================] - 4s 48ms/step - loss: 2.1844 - categorical_accuracy: 0.2021
Epoch 2/1000
30/30 [==============================] - 1s 47ms/step - loss: 1.4624 - categorical_accuracy: 0.3674
Epoch 3/1000
30/30 [==============================] - 1s 49ms/step - loss: 1.1732 - categorical_accuracy: 0.4979
Epoch 4/1000
30/30 [==============================] - 1s 47ms/step - loss: 0.7636 - categorical_accuracy: 0.6863
Epoch 5/1000
30/30 [==============================] - 1s 47ms/step - loss: 0.5567 - categorical_accuracy: 0.7442
Epoch 6/1000
30/30 [==============================] - 2s 50ms/step - loss: 0.3690 - categorical_accuracy: 0.8116
Epoch 7/1000
30/30 [==============================] - 1s 48ms/step - loss: 0.2482 - categorical_accuracy: 0.8653
Epoch 8/1000
30/30 [==============================] - 1s 47ms/step - loss: 0.5520 - categorical_accuracy: 0.7832
Epoch 9/1000
30/30 [==============================] - 1s 47ms/step - loss: 0.2125 - categorical_

In [23]:
del model

In [31]:
model.summary() # summary of our model

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 30, 64)            49408     
                                                                 
 flatten_1 (Flatten)         (None, 1920)              0         
                                                                 
 dense_3 (Dense)             (None, 64)                122944    
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 10)               

In [21]:
res = model.predict(x_test)

2/2 [==============================] - 0s 11ms/step


In [22]:
for x in range(50):
    #print(signs[np.argmax(y_test[x])] == signs[np.argmax(res[x])])
    if signs[np.argmax(y_test[x])] == signs[np.argmax(res[x])]:
        print(signs[np.argmax(y_test[x])] == signs[np.argmax(res[x])])
    elif not signs[np.argmax(y_test[x])] == signs[np.argmax(res[x])]:
        print(signs[np.argmax(y_test[x])] == signs[np.argmax(res[x])], signs[np.argmax(res[x])], signs[np.argmax(y_test[x])])


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False i love you hello
False hello i love you
True
True
True
True
True
True
True
True
False wait hi
True
True


In [17]:
print(48/50 * 100)

96.0


In [34]:
signs[np.argmax(res[0])]

'hello'

In [35]:
signs[np.argmax(y_test[0])]

'hello'

# save weights


In [36]:
model.save("model.h5") # save model

In [37]:
model = tf.keras.models.load_model("./model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite = converter.convert()

open("./model.tflite", "wb").write(tflite)

INFO:tensorflow:Assets written to: C:\Users\Baron\AppData\Local\Temp\tmp7e0q3nu9\assets


INFO:tensorflow:Assets written to: C:\Users\Baron\AppData\Local\Temp\tmp7e0q3nu9\assets


746976

In [38]:
interpreter = tf.lite.Interpreter("LiteML.tflite")

inp = interpreter.get_input_details()
out = interpreter.get_output_details()

interpreter.set_tensor(inp[0]["index"], np.expand_dims(sequences,axis=0))
interpreter.invoke()
                
                #for i in sequence[-30:]:
output_data = interpreter.get_tensor(out[0]["index"])
                
highest = np.argmax(output_data)
word = VM.get_key(signs, highest)
print(output_data.shape)
print(word)


ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_lstm_9_input:0 